In [1]:
import torch
import torch.nn as nn
import pickle
torch.manual_seed(42)
torch.set_num_threads(4)
torch.set_num_interop_threads(4)

In [2]:
if torch.cuda.is_available():
    print('GPU is available!')
else:
    print('GPU is not available.')

GPU is not available.


In [3]:

import pandas as pd

file_path1 = '../../../data/processed/ohe_25K_tracks_features_and_labels_for_training.csv'
file_path2 = '../../../data/processed/ohe_25K_tracks_features_and_labels_for_test.csv'
file_path3 = '../../../data/processed/ohe_25K_tracks_features_and_labels_for_validation.csv'
file_path4 = '../../../data/processed/ohe_25K_tracks_features_and_labels_all.csv'

training_df = pd.read_csv(file_path1)
# display(len(training_df['track_id'].unique()))
test_df = pd.read_csv(file_path2)
validation_df = pd.read_csv(file_path3)
merged_df = pd.read_csv(file_path4)

# display(len(merged_df['track_id'].unique()))
merged_df_train = merged_df[merged_df['track_id'].isin(training_df['track_id'])]
merged_df_val = merged_df[merged_df['track_id'].isin(validation_df['track_id'])]
merged_df_test = merged_df[merged_df['track_id'].isin(test_df['track_id'])]
print(merged_df_train['track_genre_top'].unique())
print(len(merged_df_train))
print(len(merged_df_val))
print(len(merged_df_test))

['Hip-Hop' 'Pop' 'Rock' 'Folk' 'Jazz' 'Electronic' 'Experimental'
 'International' 'Country' 'Old-Time / Historic' 'Soul-RnB' 'Classical'
 'Instrumental']
19744
2482
2546


In [4]:
from sklearn.preprocessing import LabelEncoder
merged_df_train_without_track_id = merged_df_train.drop('track_id',axis=1)
merged_df_train_without_track_id

label_encoder = LabelEncoder()
merged_df_train_without_track_id['target_encoded'] = label_encoder.fit_transform(merged_df_train_without_track_id['track_genre_top'])
genre_target_map = merged_df_train_without_track_id[['target_encoded', 'track_genre_top']]
merged_df_train_without_track_id.drop(['track_genre_top'], axis = 1, inplace=True)
merged_df_train_without_track_id['target_encoded'].unique()

array([ 5, 10, 11,  4,  8,  2,  3,  7,  1,  9, 12,  0,  6])

In [5]:
genre_target_map = genre_target_map.drop_duplicates(keep='last')
genre_target_map

,target_encoded,track_genre_top
20435,12,Soul-RnB
21514,9,Old-Time / Historic
22910,1,Country
24578,7,International
24652,0,Classical
24714,10,Pop
24718,8,Jazz
24752,5,Hip-Hop
24758,2,Electronic
24767,6,Instrumental


In [6]:
display(merged_df_train_without_track_id.columns)
standardize_features = ['tempo', 'rms_harmonic_mean', 'rms_harmonic_var', 'rms_percussive_mean',
       'rms_percussive_var', 'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_rolloff_mean', 'spectral_rolloff_var', 'zero_crossings',
       'chroma_stft_0_mean', 'chroma_stft_1_mean', 'chroma_stft_2_mean',
       'chroma_stft_3_mean', 'chroma_stft_4_mean', 'chroma_stft_5_mean',
       'chroma_stft_6_mean', 'chroma_stft_7_mean', 'chroma_stft_8_mean',
       'chroma_stft_9_mean', 'chroma_stft_10_mean', 'chroma_stft_11_mean',
       'chroma_stft_0_var', 'chroma_stft_1_var', 'chroma_stft_2_var',
       'chroma_stft_3_var', 'chroma_stft_4_var', 'chroma_stft_5_var',
       'chroma_stft_6_var', 'chroma_stft_7_var', 'chroma_stft_8_var',
       'chroma_stft_9_var', 'chroma_stft_10_var', 'chroma_stft_11_var',
       'MFCC_1_mean', 'MFCC_2_mean', 'MFCC_3_mean', 'MFCC_4_mean',
       'MFCC_5_mean', 'MFCC_6_mean', 'MFCC_7_mean', 'MFCC_8_mean',
       'MFCC_9_mean', 'MFCC_10_mean', 'MFCC_11_mean', 'MFCC_12_mean',
       'MFCC_13_mean', 'MFCC_14_mean', 'MFCC_15_mean', 'MFCC_16_mean',
       'MFCC_17_mean', 'MFCC_18_mean', 'MFCC_19_mean', 'MFCC_20_mean',
       'MFCC_1_var', 'MFCC_2_var', 'MFCC_3_var', 'MFCC_4_var', 'MFCC_5_var',
       'MFCC_6_var', 'MFCC_7_var', 'MFCC_8_var', 'MFCC_9_var', 'MFCC_10_var',
       'MFCC_11_var', 'MFCC_12_var', 'MFCC_13_var', 'MFCC_14_var',
       'MFCC_15_var', 'MFCC_16_var', 'MFCC_17_var', 'MFCC_18_var',
       'MFCC_19_var', 'MFCC_20_var', 'key_A', 'key_Ab', 'key_B', 'key_Bb',
       'key_C', 'key_C#', 'key_D', 'key_E', 'key_Eb', 'key_F', 'key_F#',
       'key_G', 'scale_major', 'scale_minor']

Index(['Unnamed: 0', 'track_title', 'artist_name', 'set_split', 'set_subset',
       'tempo', 'rms_harmonic_mean', 'rms_harmonic_var', 'rms_percussive_mean',
       'rms_percussive_var', 'spectral_centroid_mean', 'spectral_centroid_var',
       'spectral_rolloff_mean', 'spectral_rolloff_var', 'zero_crossings',
       'chroma_stft_0_mean', 'chroma_stft_1_mean', 'chroma_stft_2_mean',
       'chroma_stft_3_mean', 'chroma_stft_4_mean', 'chroma_stft_5_mean',
       'chroma_stft_6_mean', 'chroma_stft_7_mean', 'chroma_stft_8_mean',
       'chroma_stft_9_mean', 'chroma_stft_10_mean', 'chroma_stft_11_mean',
       'chroma_stft_0_var', 'chroma_stft_1_var', 'chroma_stft_2_var',
       'chroma_stft_3_var', 'chroma_stft_4_var', 'chroma_stft_5_var',
       'chroma_stft_6_var', 'chroma_stft_7_var', 'chroma_stft_8_var',
       'chroma_stft_9_var', 'chroma_stft_10_var', 'chroma_stft_11_var',
       'MFCC_1_mean', 'MFCC_2_mean', 'MFCC_3_mean', 'MFCC_4_mean',
       'MFCC_5_mean', 'MFCC_6_mean', 'MFCC_7_

In [7]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
merged_df_train_without_track_id[standardize_features] = scaler.fit_transform(merged_df_train_without_track_id[standardize_features])
merged_df_std=merged_df_train_without_track_id[standardize_features + ['target_encoded']]
merged_df_std

,tempo,rms_harmonic_mean,rms_harmonic_var,rms_percussive_mean,rms_percussive_var,spectral_centroid_mean,spectral_centroid_var,spectral_rolloff_mean,spectral_rolloff_var,zero_crossings,...,key_C#,key_D,key_E,key_Eb,key_F,key_F#,key_G,scale_major,scale_minor,target_encoded
0,1.864288,-1.035443,-0.579581,0.957190,1.560255,1.536069,1.134753,1.408480,0.136749,1.679187,...,-0.214827,-0.346463,-0.350028,-0.210665,-0.31643,-0.214310,-0.338431,-0.938573,0.938573,5
1,-1.719729,-0.932115,-0.414481,0.050343,0.006433,1.219214,0.773509,1.137737,0.128330,1.343175,...,-0.214827,-0.346463,-0.350028,-0.210665,-0.31643,-0.214310,2.954810,-0.938573,0.938573,5
2,-1.089572,-0.783258,-0.155933,0.538267,0.601320,0.637687,0.630009,0.901332,0.268890,0.259369,...,4.654911,-0.346463,-0.350028,-0.210665,-0.31643,-0.214310,-0.338431,-0.938573,0.938573,5
3,-0.473115,-0.029213,-0.581031,0.430372,0.400442,0.576400,-0.683018,0.188259,-0.761162,1.361202,...,-0.214827,-0.346463,-0.350028,-0.210665,-0.31643,4.666131,-0.338431,1.065448,-1.065448,10
4,0.651651,-0.666152,-0.444718,0.761369,0.588985,0.663386,1.054137,0.680106,0.760133,0.604463,...,-0.214827,-0.346463,-0.350028,-0.210665,-0.31643,-0.214310,-0.338431,1.065448,-1.065448,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24767,0.035708,-1.336052,-0.713112,-1.332951,-0.761610,-1.226894,-0.911289,-1.264413,-0.963222,-1.082376,...,-0.214827,2.886315,-0.350028,-0.210665,-0.31643,-0.214310,-0.338431,1.065448,-1.065448,6
24768,0.328281,0.019316,0.088518,-0.727245,-0.519202,-0.917796,-0.513181,-0.889147,-0.229800,-1.025007,...,-0.214827,-0.346463,-0.350028,-0.210665,-0.31643,-0.214310,2.954810,1.065448,-1.065448,4
24769,-0.900525,-0.286221,0.272580,-1.062523,-0.581863,-1.380695,-0.300337,-1.373377,-0.079032,-1.042933,...,-0.214827,-0.346463,-0.350028,-0.210665,-0.31643,-0.214310,-0.338431,1.065448,-1.065448,4
24770,-0.230268,-0.704168,-0.290770,-0.621192,-0.236322,0.669120,2.811872,0.696052,2.731086,0.360755,...,-0.214827,-0.346463,-0.350028,-0.210665,-0.31643,-0.214310,-0.338431,-0.938573,0.938573,3


In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, classification_report
from music_rnn import MusicRNN

# Hyperparameters
input_size = 88
hidden_size = 64
output_size = 13
num_layers = 2
dropout = 0.5
learning_rate = 0.001
num_epochs = 5


# Initialize the model
music_rnn_model = MusicRNN(input_size, hidden_size, output_size, num_layers, dropout)
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(music_rnn_model.parameters(), lr=learning_rate)

In [ ]:
torch.manual_seed(42)
# Train the model

for epoch in range(num_epochs):
    epoch_loss = 0
    music_rnn_model.train()  # Set the model to training mode

    for _, batch in merged_df_std.iterrows():
        optimizer.zero_grad()

        # Assuming batch is a pandas Series
        features = torch.tensor(batch.iloc[:-1].astype(float).values, dtype=torch.float32)
        print("feature1:",features.shape)
        print(features)

        # Reshape features to have dimensions (1, sequence_length, input_size)
        features = features.view(1, -1, features.size(-1))
        # print("feature2:",features.shape)
        # print(features)

        label = torch.tensor([int(batch.iloc[-1])], dtype=torch.long)
        # print("label:",label.shape)

        predictions = music_rnn_model(features)
        # print("predictions:",predictions.shape)

        loss = criterion(predictions, label)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f'Epoch: {epoch + 1}, Train Loss: {epoch_loss / len(merged_df_std)}')
# The running may take around 10 minutes to reach the results.

In [10]:
# prepare validation dataset
from sklearn.preprocessing import LabelEncoder
merged_df_val_without_track_id = merged_df_val.drop('track_id',axis=1)

label_encoder = LabelEncoder()
merged_df_val_without_track_id['target_encoded'] = label_encoder.fit_transform(merged_df_val_without_track_id['track_genre_top'])
merged_df_val_without_track_id.drop(['track_genre_top'],axis = 1,inplace=True)
merged_df_val_without_track_id
merged_df_val_without_track_id['target_encoded'].unique()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
merged_df_val_without_track_id[standardize_features] = scaler.fit_transform(merged_df_val_without_track_id[standardize_features])
merged_df_val_std=merged_df_val_without_track_id[standardize_features + ['target_encoded']]
merged_df_val_std

,tempo,rms_harmonic_mean,rms_harmonic_var,rms_percussive_mean,rms_percussive_var,spectral_centroid_mean,spectral_centroid_var,spectral_rolloff_mean,spectral_rolloff_var,zero_crossings,...,key_C#,key_D,key_E,key_Eb,key_F,key_F#,key_G,scale_major,scale_minor,target_encoded
9,-0.210394,3.212031,0.734023,1.639035,0.594010,-0.344541,-0.315683,-0.300847,0.135001,0.002292,...,-0.219413,-0.341334,-0.36017,-0.226383,-0.333184,-0.195088,-0.329440,1.092053,-1.092053,3
47,1.471329,1.338846,-0.389052,0.802374,0.186905,0.102743,-0.740821,-0.103594,-0.791763,0.082751,...,-0.219413,-0.341334,-0.36017,-0.226383,3.001344,-0.195088,-0.329440,-0.915707,0.915707,11
48,-1.269256,2.079879,-0.094912,0.808200,-0.286896,-0.037556,-0.286739,0.203055,0.202868,-0.575510,...,-0.219413,-0.341334,-0.36017,-0.226383,-0.333184,-0.195088,3.035456,1.092053,-1.092053,11
60,3.056952,-0.158298,1.847371,0.459609,0.484610,-0.644874,0.726037,-0.851007,0.735984,-0.406455,...,-0.219413,-0.341334,-0.36017,-0.226383,3.001344,-0.195088,-0.329440,-0.915707,0.915707,5
61,-1.269256,0.719850,0.396057,0.292676,0.041333,-0.802691,-0.102332,-0.655532,0.620223,-1.210341,...,-0.219413,-0.341334,-0.36017,-0.226383,3.001344,-0.195088,-0.329440,1.092053,-1.092053,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24601,-0.210394,-0.897091,-0.517462,-0.061639,0.366116,-0.485215,1.291531,-0.328954,1.940324,-0.795561,...,-0.219413,-0.341334,-0.36017,-0.226383,-0.333184,-0.195088,-0.329440,1.092053,-1.092053,2
24602,-0.210394,-0.907546,-0.640837,-0.025777,-0.173597,-0.188922,0.339351,0.164228,0.885298,-0.466501,...,-0.219413,-0.341334,-0.36017,-0.226383,3.001344,-0.195088,-0.329440,-0.915707,0.915707,2
24603,-1.435648,-0.437296,-0.546000,1.164415,2.923304,-0.330475,-0.219239,-0.052997,0.091928,-0.587084,...,-0.219413,-0.341334,-0.36017,-0.226383,-0.333184,-0.195088,-0.329440,-0.915707,0.915707,2
24604,0.361392,-0.705487,-0.464852,-0.117331,0.692785,0.070072,2.794259,0.233138,3.643704,-0.654565,...,-0.219413,-0.341334,-0.36017,-0.226383,-0.333184,-0.195088,-0.329440,-0.915707,0.915707,2


In [15]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


# validate the model
for epoch in range(num_epochs):
    music_rnn_model.eval()  # Set the model to evaluation mode
    eval_loss = 0
    correct_predictions = 0

    true_labels = []  # List to collect true labels
    predicted_labels = []  # List to collect predicted labels

    with torch.no_grad():
        for _, eval_batch in merged_df_val_std.iterrows():
            eval_features = torch.tensor(eval_batch.iloc[:-1].astype(float).values, dtype=torch.float32)
            eval_features = eval_features.view(1, -1, eval_features.size(-1))

            eval_label = torch.tensor([int(eval_batch.iloc[-1])], dtype=torch.long)
            eval_predictions = music_rnn_model(eval_features)
            eval_label = eval_label.expand_as(eval_predictions[:, 0])

            eval_loss += criterion(eval_predictions, eval_label).item()

            # Collect true and predicted labels for accuracy calculation
            true_labels.extend(eval_label.numpy().tolist())
            _, predicted_classes = torch.max(eval_predictions, 1)
            predicted_labels.extend(predicted_classes.numpy())

            # Calculate correct predictions
            correct_predictions += (predicted_classes == eval_label).sum().item()

    eval_accuracy = accuracy_score(true_labels, predicted_labels)
    eval_recall = recall_score(true_labels, predicted_labels, average='weighted')
    eval_precision = precision_score(true_labels, predicted_labels, average='weighted')
    eval_f1 = f1_score(true_labels, predicted_labels, average='weighted')
    average_eval_loss = eval_loss / len(merged_df_val_std)  # Number of samples used for evaluation

    print(f'Epoch: {epoch + 1}, Evaluation Loss: {average_eval_loss}, Evaluation Accuracy: {eval_accuracy * 100}%')
    print(f'Epoch: {epoch + 1}, Evaluation Loss: {average_eval_loss}, Evaluation Recall: {eval_recall * 100}%')
    print(f'Epoch: {epoch + 1}, Evaluation Loss: {average_eval_loss}, Evaluation Precision: {eval_precision * 100}%')
    print(f'Epoch: {epoch + 1}, Evaluation Loss: {average_eval_loss}, Evaluation F1: {eval_f1 * 100}%')
 
    # Display classification report
    # report = classification_report(true_labels, predicted_labels)
    # print(f'Classification Report:\n{report}')
    # conf_matrix = confusion_matrix(true_labels, predicted_labels)
    # print(f'Confusion Matrix:\n{conf_matrix}')



/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1, Evaluation Loss: 1.3186032157909129, Evaluation Accuracy: 58.9041095890411%
Epoch: 1, Evaluation Loss: 1.3186032157909129, Evaluation Recall: 58.9041095890411%
Epoch: 1, Evaluation Loss: 1.3186032157909129, Evaluation Precision: 55.229962811877876%
Epoch: 1, Evaluation Loss: 1.3186032157909129, Evaluation F1: 54.441470662009685%


/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 2, Evaluation Loss: 1.3186032157909129, Evaluation Accuracy: 58.9041095890411%
Epoch: 2, Evaluation Loss: 1.3186032157909129, Evaluation Recall: 58.9041095890411%
Epoch: 2, Evaluation Loss: 1.3186032157909129, Evaluation Precision: 55.229962811877876%
Epoch: 2, Evaluation Loss: 1.3186032157909129, Evaluation F1: 54.441470662009685%


/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 3, Evaluation Loss: 1.3186032157909129, Evaluation Accuracy: 58.9041095890411%
Epoch: 3, Evaluation Loss: 1.3186032157909129, Evaluation Recall: 58.9041095890411%
Epoch: 3, Evaluation Loss: 1.3186032157909129, Evaluation Precision: 55.229962811877876%
Epoch: 3, Evaluation Loss: 1.3186032157909129, Evaluation F1: 54.441470662009685%


/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 4, Evaluation Loss: 1.3186032157909129, Evaluation Accuracy: 58.9041095890411%
Epoch: 4, Evaluation Loss: 1.3186032157909129, Evaluation Recall: 58.9041095890411%
Epoch: 4, Evaluation Loss: 1.3186032157909129, Evaluation Precision: 55.229962811877876%
Epoch: 4, Evaluation Loss: 1.3186032157909129, Evaluation F1: 54.441470662009685%
Epoch: 5, Evaluation Loss: 1.3186032157909129, Evaluation Accuracy: 58.9041095890411%
Epoch: 5, Evaluation Loss: 1.3186032157909129, Evaluation Recall: 58.9041095890411%
Epoch: 5, Evaluation Loss: 1.3186032157909129, Evaluation Precision: 55.229962811877876%
Epoch: 5, Evaluation Loss: 1.3186032157909129, Evaluation F1: 54.441470662009685%


/Users/1108232/PycharmProjects/MADS-699-Capstone-fa23-team12/venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
# save model
torch.save(music_rnn_model.state_dict(), '../../../models/final_models/lstm_genre_prediction.pth')

In [21]:
import shap
import numpy as np

X_train = merged_df_std.iloc[:, :-1].to_numpy()
X_test = merged_df_val_std.iloc[:, :-1].to_numpy()


In [22]:
print(type(X_train))
print(type(X_test))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
# to get feature importances
def music_rnn_predict(data):
    # Convert the input data to a PyTorch tensor
    # Reshape data to (num_samples, 1, num_features)
    data_tensor = torch.tensor(data, dtype=torch.float32).reshape(-1, 1, 88)

    # Move the tensor to the same device as the model
    device = next(music_rnn_model.parameters()).device
    data_tensor = data_tensor.to(device)

    # Set the model to evaluation mode and make predictions
    music_rnn_model.eval()
    with torch.no_grad():
        predictions = music_rnn_model(data_tensor)
    
    # Return the predictions as a numpy array
    return predictions.cpu().numpy()

# Reshape background data
background = X_train[np.random.choice(X_train.shape[0], 1000, replace=False)]

# Create a SHAP Explainer
explainer = shap.KernelExplainer(music_rnn_predict, background)

# Reshape sample data and calculate SHAP values
sample = X_test[np.random.choice(X_test.shape[0], 100, replace=False)]
shap_values = explainer(sample)


In [33]:
feature_importances_per_class = {}

# Iterate over each class (label)
for label_index in range(shap_values.shape[2]):
    # Calculate mean absolute SHAP values for each feature for this label
    mean_abs_shap_per_feature = np.abs(shap_values.values[:, :, label_index]).mean(axis=0)
    
    # Assuming 'feature_cols' is a list of your feature names
    feature_importances_per_class[f'Label_{label_index}'] = dict(zip(standardize_features, mean_abs_shap_per_feature))

# Convert the dictionary to a DataFrame for easier handling
df_feature_importances = pd.DataFrame(feature_importances_per_class)



In [35]:
lstm_fi_df = pd.DataFrame(df_feature_importances.T.mean()).T
lstm_fi_df.to_csv('../../../data/resources/lstm_feature_importances.csv')

In [36]:
lstm_fi_df

,tempo,rms_harmonic_mean,rms_harmonic_var,rms_percussive_mean,rms_percussive_var,spectral_centroid_mean,spectral_centroid_var,spectral_rolloff_mean,spectral_rolloff_var,zero_crossings,...,key_C,key_C#,key_D,key_E,key_Eb,key_F,key_F#,key_G,scale_major,scale_minor
0,0.033103,0.044589,0.068935,0.087963,0.083679,0.053524,0.112394,0.066553,0.049029,0.094978,...,0.058404,0.049227,0.054089,0.112123,0.042466,0.041516,0.046979,0.052697,0.025251,0.029463
